<a href="https://colab.research.google.com/github/Ashonet/S-P100_ANALYSIS/blob/main/SP100_Analysis_1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install forex-python

In [ ]:
from forex_python.converter import CurrencyRates

def get_currency_price(base_currency, target_currency):
    c = CurrencyRates()
    price = c.get_rate(base_currency, target_currency)
    return price

# Example: Get the current price of USD to EUR
base_currency = 'USD'
target_currency = 'EUR'

current_price = get_currency_price(base_currency, target_currency)
print(f"The current price of {base_currency} to {target_currency} is: {current_price}")

The current price of USD to EUR is: 0.9372071227741331


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def get_sp100_companies():
    sp100_symbols = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
    sp100_symbols = sp100_symbols['Symbol'].tolist()
    return sp100_symbols

def get_intrinsic_value(symbol):
    try:
        data = yf.Ticker(symbol)
        dividends = data.dividends

        if len(dividends) < 2:
            return "Insufficient data for analysis", None, None  # Return both message and None value if insufficient data

        growth_rate = (dividends.iloc[-1] - dividends.iloc[0]) / dividends.iloc[0]  # Calculate growth rate based on dividends

        latest_dividend = dividends.iloc[-1]
        discount_rate = 0.1  # Set your required rate of return

        # Ensure growth rate is not greater than the discount rate to avoid issues
        growth_rate = min(growth_rate, discount_rate * 0.9)  # Set a maximum growth rate

        intrinsic_value = latest_dividend / (discount_rate - growth_rate)
        current_price = data.history(period="1d")['Close'][0]

        hold_range_percentage = 5
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None


def analyze_stocks(stock_symbols, start_date, end_date):
    recommendations = {}
    for symbol in stock_symbols:
        stock_data = get_stock_data(symbol, start_date, end_date)
        if not stock_data.empty:
            try:
                recommendation, intrinsic_value, price_type = get_intrinsic_value(symbol)
                if recommendation is not None and intrinsic_value is not None and price_type is not None:
                    if recommendation == "Buy":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                    elif recommendation == "Sell":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                    elif recommendation == "Hold":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value[0]:.2f} to {intrinsic_value[1]:.2f}"
                else:
                    recommendations[symbol] = "Unable to evaluate"
            except Exception as e:
                print(f"Error analyzing {symbol}: {e}")
                recommendations[symbol] = "Error in analysis"
        else:
            recommendations[symbol] = "No data available"
    return recommendations

# Get S&P 100 company symbols
sp100_symbols = get_sp100_companies()

# Example usage:
start_date = '2019-12-01'
end_date = '2023-12-20'

# Analyze stocks from S&P 100
stock_recommendations = analyze_stocks(sp100_symbols, start_date, end_date)

# Output the formatted recommendations
print("--------")
for symbol, recommendation in stock_recommendations.items():
    print(f"{symbol}: {recommendation}")
print("--------")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def get_sp100_companies():
    sp100_symbols = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
    sp100_symbols = sp100_symbols['Symbol'].tolist()
    return sp100_symbols

def get_intrinsic_value(symbol):
    try:
        data = yf.Ticker(symbol)
        historical_data = data.history(period="max")

        if historical_data.empty:
            return "Insufficient data for analysis", None, None

        earnings_per_share = data.info.get('trailingEps', None)
        book_value_per_share = data.info.get('bookValue', None)

        if earnings_per_share is None or np.isnan(earnings_per_share) or book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None, None
        # Relative Value
        intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)
        current_price = historical_data['Close'][-1]

        if np.isnan(intrinsic_value):
            return "Unable to calculate intrinsic value", None, None

        hold_range_percentage = 5
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None




def analyze_stocks(stock_symbols, start_date, end_date):
    recommendations = {}
    for symbol in stock_symbols:
        stock_data = get_stock_data(symbol, start_date, end_date)
        if not stock_data.empty:
            try:
                recommendation, intrinsic_value, price_type = get_intrinsic_value(symbol)
                if recommendation is not None and intrinsic_value is not None and price_type is not None:
                    if recommendation == "Buy":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                    elif recommendation == "Sell":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                    elif recommendation == "Hold":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value[0]:.2f} to {intrinsic_value[1]:.2f}"
                else:
                    recommendations[symbol] = "Unable to evaluate"
            except Exception as e:
                print(f"Error analyzing {symbol}: {e}")
                recommendations[symbol] = "Error in analysis"
        else:
            recommendations[symbol] = "No data available"
    return recommendations

# Get S&P 100 company symbols
sp100_symbols = get_sp100_companies()

# Example usage:
start_date = '2019-12-01'
end_date = '2023-12-20'

# Analyze stocks from S&P 100
stock_recommendations = analyze_stocks(sp100_symbols, start_date, end_date)

# Output the formatted recommendations
print("--------")
for symbol, recommendation in stock_recommendations.items():
    print(f"{symbol}: {recommendation}")
print("--------")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def get_sp100_companies():
    sp100_symbols = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
    sp100_symbols = sp100_symbols['Symbol'].tolist()
    return sp100_symbols

def calculate_residual_income(book_value, cost_of_equity, future_residual_income):
    # Calculate intrinsic value using the residual income model
    intrinsic_value = book_value * (1 + (future_residual_income * 0.01) / (1 + cost_of_equity * 0.01))
    return intrinsic_value

def estimate_future_residual_income(symbol):
    try:
        data = yf.Ticker(symbol)
        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None

        # Fetch historical data (assuming quarterly data)
        historical_data = data.history(period="1y")  # Adjust period as needed

        if historical_data.empty or len(historical_data) < 4:  # At least 4 quarters needed for trend estimation
            return "Insufficient historical data for analysis", None

        # Calculate past residual incomes based on historical book values
        historical_book_values = historical_data['Close'].to_list()  # Assuming 'Close' prices represent book values
        past_residual_incomes = [0]  # Initialize with zero as the starting residual income

        for i in range(1, len(historical_book_values)):
            residual_income = max(0, historical_book_values[i] - book_value_per_share)
            past_residual_incomes.append(residual_income)

        # Estimate growth rate based on past performance (simple average growth rate)
        average_growth_rate = sum(past_residual_incomes[-4:]) / len(past_residual_incomes[-4:])

        # Project future residual income using the average growth rate
        future_residual_income = past_residual_incomes[-1] + average_growth_rate

        return future_residual_income
    except Exception as e:
        print(f"Error estimating future residual income for {symbol}: {e}")
        return None

def get_beta(symbol):
    try:
        stock_data = yf.Ticker(symbol)
        stock_history = stock_data.history(period='max')

        market_index = '^GSPC'  # S&P 500 index
        market_data = yf.Ticker(market_index).history(period='max')

        cov_stock_market = stock_history['Close'].pct_change().cov(market_data['Close'].pct_change())
        var_market = market_data['Close'].pct_change().var()

        beta = cov_stock_market / var_market
        return beta
    except Exception as e:
        print(f"Error fetching beta for {symbol}: {e}")
        return None

def calculate_cost_of_equity(symbol):
    try:
        risk_free_rate = 0.04  # Replace with actual risk-free rate (e.g., 10-year Treasury yield)
        market_return = 0.08  # Replace with expected market return

        beta = get_beta(symbol)
        if beta is None:
            return None

        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
        return cost_of_equity
    except Exception as e:
        print(f"Error calculating cost of equity for {symbol}: {e}")
        return None

def get_intrinsic_value_residual_income(symbol, cost_of_equity):
    try:
        data = yf.Ticker(symbol)
        historical_data = data.history(period="max")

        if historical_data.empty:
            return "Insufficient data for analysis", None, None

        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None, None

        # Calculate future residual income based on historical performance
        future_residual_income = estimate_future_residual_income(symbol)

        if future_residual_income is None:
            return "Unable to estimate future residual income", None, None

        # Call the function to calculate intrinsic value using estimated future residual income
        intrinsic_value = calculate_residual_income(book_value_per_share, cost_of_equity, future_residual_income)
        current_price = historical_data['Close'][-1]

        if np.isnan(intrinsic_value):
            return "Unable to calculate intrinsic value", None, None

        hold_range_percentage = 7
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None

def analyze_stocks(stock_symbols, start_date, end_date):
    recommendations = {}
    for symbol in stock_symbols:
        stock_data = get_stock_data(symbol, start_date, end_date)
        if not stock_data.empty:
            try:
                cost_of_equity = calculate_cost_of_equity(symbol)
                if cost_of_equity is None:
                    recommendations[symbol] = "Unable to estimate cost of equity"
                    continue

                recommendation, intrinsic_value, price_type = get_intrinsic_value_residual_income(symbol, cost_of_equity)
                if recommendation is not None and price_type is not None:
                    if intrinsic_value is None:
                        recommendations[symbol] = f"Unable to calculate intrinsic value"
                    else:
                        if recommendation == "Buy":
                            recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                        elif recommendation == "Sell":
                            recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                        elif recommendation == "Hold":
                            recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value[0]:.2f} to {intrinsic_value[1]:.2f}"
                else:
                    recommendations[symbol] = "Unable to evaluate"
            except Exception as e:
                print(f"Error analyzing {symbol}: {e}")
                recommendations[symbol] = "Error in analysis"
        else:
            recommendations[symbol] = "No data available"
    return recommendations

# Get S&P 100 company symbols
sp100_symbols = get_sp100_companies()

# Example usage:
start_date = '2019-12-01'
end_date = '2023-12-20'

# Analyze stocks from S&P 100
stock_recommendations = analyze_stocks(sp100_symbols, start_date, end_date)

# Output the formatted recommendations
print("--------")
for symbol, recommendation in stock_recommendations.items():
    print(f"{symbol}: {recommendation}")
print("--------")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf
import requests

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def get_sp100_companies():
    sp100_symbols = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
    sp100_symbols = sp100_symbols['Symbol'].tolist()
    return sp100_symbols

def get_growth_rate_from_prices(symbol):
    try:
        # Retrieve historical stock prices directly from Yahoo Finance API
        url = f"https://query1.finance.yahoo.com/v7/finance/download/{symbol}?period1=1420070400&period2=1672531200&interval=1mo&events=history"
        response = requests.get(url)

        if response.status_code != 200:
            return "Failed to fetch data for growth rate analysis", None

        stock_data = pd.read_csv(io.StringIO(response.text))

        # Calculate growth rate based on historical stock prices
        stock_data['Date'] = pd.to_datetime(stock_data['Date'])
        stock_data.set_index('Date', inplace=True)
        stock_data['Earnings'] = stock_data['Close']
        earnings_growth = stock_data['Earnings'].pct_change()
        average_growth_rate = earnings_growth.mean()

        if np.isnan(average_growth_rate):
            return "Unable to calculate growth rate", None

        return "Success", average_growth_rate
    except Exception as e:
        print(f"Error fetching growth rate for {symbol}: {e}")
        return None, None


def get_intrinsic_value(symbol):
    try:
        data = yf.Ticker(symbol)
        historical_data = data.history(period="max")

        if historical_data.empty:
            return "Insufficient data for analysis", None, None

        free_cash_flow = data.info.get('freeCashflow', None)
        growth_status, growth_rate = get_growth_rate_from_prices(symbol)
        discount_rate = 0.10  # Assumed discount rate for demonstration purposes

        if free_cash_flow is None or np.isnan(free_cash_flow) or growth_status != "Success" or growth_rate is None:
            return "Insufficient or invalid financial data for analysis", None, None

        intrinsic_value = free_cash_flow * (1 + growth_rate) / (discount_rate - growth_rate)
        current_price = historical_data['Close'][-1]

        if np.isnan(intrinsic_value):
            return "Unable to calculate intrinsic value", None, None

        hold_range_percentage = 5
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None




def analyze_stocks(stock_symbols, start_date, end_date):
    recommendations = {}
    for symbol in stock_symbols:
        stock_data = get_stock_data(symbol, start_date, end_date)
        if not stock_data.empty:
            try:
                recommendation, intrinsic_value, price_type = get_intrinsic_value(symbol)
                if recommendation is not None and intrinsic_value is not None and price_type is not None:
                    if recommendation == "Buy":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                    elif recommendation == "Sell":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value:.2f}"
                    elif recommendation == "Hold":
                        recommendations[symbol] = f"{recommendation} - {price_type} - {intrinsic_value[0]:.2f} to {intrinsic_value[1]:.2f}"
                else:
                    recommendations[symbol] = "Unable to evaluate"
            except Exception as e:
                print(f"Error analyzing {symbol}: {e}")
                recommendations[symbol] = "Error in analysis"
        else:
            recommendations[symbol] = "No data available"
    return recommendations

# Get S&P 100 company symbols
sp100_symbols = get_sp100_companies()

# Example usage:
start_date = '2019-12-01'
end_date = '2023-12-20'

# Analyze stocks from S&P 100
stock_recommendations = analyze_stocks(sp100_symbols, start_date, end_date)

# Output the formatted recommendations
print("--------")
for symbol, recommendation in stock_recommendations.items():
    print(f"{symbol}: {recommendation}")
print("--------")

intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)